#### **利用ogr开源库进行矢量文件的读写**


In [2]:
from osgeo import ogr

##### **1. 用python读入矢量文件**

In [3]:
path_kunming = "E:/YNU/5/OpenSourceGIS/Assignment_3/data/矢量数据/kumming_polygon.shp"

In [4]:
### 读入矢量文件
ds = ogr.Open(path_kunming, 0)    # 0是只读，1是可写
layer = ds.GetLayer(0)            # 获取矢量文件Layer
num_fea = layer.GetFeatureCount()   ## 统计要素个数
print('Number of feature: ', num_fea)
num_field = layer.GetFeature(0).GetFieldCount()   ## 统计属性个数
print('Number of fields:', num_field)

Number of feature:  14
Number of fields: 32


e:\anaconda3\envs\gisenv\Lib\site-packages\osgeo\ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


##### **2. 查看矢量文件属性字段**

In [5]:
### 查看属性字段名
fields = []
fea = layer.GetFeature(0)
for i in range(0, num_field):
    field = fea.GetFieldDefnRef(i).name
    fields.append(field)
print(fields)


['OBJECTID', '地名', '区划码', '县级', '县级码', '县级类', '地级', '地级码', '地级类', '省级', '省级码', '省级类', '曾用名', '备注', 'ENG_NAME', 'VAR_NAME', 'code', 'NAME_3', 'VAR_NAME3', 'GID_3', 'TYPE_3', 'NAME_2', 'VAR_NAME2', 'GID_2', 'TYPE_2', 'NAME_1', 'VAR_NAME1', 'GID_1', 'TYPE_1', 'year', 'Shape_Leng', 'Shape_Area']


In [6]:
### 查看属性字段值（以地名为例）
field_name = '地名'
fea = layer.GetFeature(0)
### 获得所有要素的字段值
for i in range(layer.GetFeatureCount()):   ##
  fea = layer.GetFeature(i)
  print('Filed value:', fea.GetField(field_name))

Filed value: 安宁市
Filed value: 呈贡区
Filed value: 东川区
Filed value: 富民县
Filed value: 官渡区
Filed value: 晋宁区
Filed value: 禄劝彝族苗族自治县
Filed value: 盘龙区
Filed value: 石林彝族自治县
Filed value: 嵩明县
Filed value: 五华区
Filed value: 西山区
Filed value: 寻甸回族彝族自治县
Filed value: 宜良县


In [7]:
ds = None

##### **3. 写出矢量文件**

In [8]:
path_kunming_out_1 = 'data/Section-4/kunming_districts_dtname_anning.shp'
path_kunming_out_2 = 'data/Section-4/kunming_chenggong_anning.shp'


##### 3.1 写出只含有特定字段的矢量文件
1）读入矢量数据并获取矢量数据图层；  
2）创建新矢量文件，以此创建数据源、图层、字段。其中新建图层需定义图层名、几何、空间参考。新建字段需先定义字段，定义字段需提供字段名、字段数据类型。  
3）遍历矢量图层要素，写入要素几何及所需属性字段。


In [9]:
field_name = 'dt_name'
in_ds = ogr.Open(path_kunming, 0)    # 0是只读，1是可写
in_layer = in_ds.GetLayer()             # 获取矢量文件图层Layer
## 定义写出文件
driver = ogr.GetDriverByName('ESRI Shapefile')   ## 文件驱动（用于写出文件）
ds_out = driver.CreateDataSource(path_kunming_out_1)  ## 创建数据源DataSource
layer_out = ds_out.CreateLayer('kunming_districts_dtname', geom_type=ogr.wkbPolygon, srs=in_layer.GetSpatialRef())
field_defn = ogr.FieldDefn(field_name, ogr.OFTString)   ## 定义属性字段
layer_out.CreateField(field_defn)    ## 在写出图层中新建属性字段
# fea_defn = layer_out.GetFeature(0).GetDefnRef()    ## layer_out没有要素，故.GetFeature(0)会报错
fea_defn = layer_out.GetLayerDefn()  ## 获得要素定义（即图层定义）
### 创建要素（将复制‘昆明市边界_wgs84.shp’文件中要素)
for i in range(layer.GetFeatureCount()):   ##
    in_fea = in_layer.GetFeature(i)
    in_geo = in_fea.geometry()             ## 获得原矢量要素的几何
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(in_geo)
    field_value = in_fea.GetField(field_name)
    print(field_value)
    fea_out.SetField(0, field_value)
    layer_out.CreateFeature(fea_out)
ds_out = None         #  ### 保存/关闭 DataSource (!重要)



AttributeError: 'NoneType' object has no attribute 'CreateField'

##### 3.2 写出只含有呈贡区的矢量文件


In [10]:
in_ds = ogr.Open(path_kunming, 0)           # 0是只读，1是可写
in_layer = in_ds.GetLayer()    # 获取矢量文件Layer
### 创建写出文件
driver = ogr.GetDriverByName('ESRI Shapefile')
ds_out = driver.CreateDataSource(path_kunming_out_2)
layer_out = ds_out.CreateLayer('kunming_chenggong', geom_type=ogr.wkbPolygon, srs=in_layer.GetSpatialRef())
fea_defn = layer_out.GetLayerDefn()    ## 获得空要素定义(或空图层定义)

### 创建字段
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (before field creating): ', layer_defn.GetFieldCount())  ### 字段创建前字段数
in_fea = in_layer.GetFeature(0)
for i in range(in_fea.GetFieldCount()):
  field_defn = in_fea.GetFieldDefnRef(i)
  layer_out.CreateField(field_defn)
layer_defn = layer_out.GetLayerDefn()
print('Number of fields (after field creating): ', layer_defn.GetFieldCount())

### 创建要素（复制‘昆明市边界_wgs84.shp’文件中要素)
for i in range(in_layer.GetFeatureCount()):   ##
  in_fea = layer.GetFeature(i)
  if in_fea.GetField('dt_name') == '呈贡区' or in_fea.GetField('dt_name') == '安宁市':
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(in_fea.geometry())
    for i in range(in_fea.GetFieldCount()):
      field_value = in_fea.GetField(i)
      fea_out.SetField(i, field_value)
    layer_out.CreateFeature(fea_out)
ds_out = None        #  ### 保存/关闭 DataSource (重要)



AttributeError: 'NoneType' object has no attribute 'GetLayerDefn'

##### 课后练习：写出只含五华区、盘龙区、西山区、官渡区、呈贡区的矢量文件。